# **Program to download and pre-process North Dakota well data for sub-county analysis**

# **1.a  Download North Dakota monthly production data from DMR ~10min**



In [13]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [14]:
# create list of all yyyy-mm for data download (Note, May 2015 is the first month ND data is available in xlxs)
myList = pd.date_range('2015-04-10',datetime.today()- relativedelta(months=3), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format

# Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
website = "https://www.dmr.nd.gov/oilgas/mpr/"

In [15]:
# Review the list of monthly data
myList

['2015_05',
 '2015_06',
 '2015_07',
 '2015_08',
 '2015_09',
 '2015_10',
 '2015_11',
 '2015_12',
 '2016_01',
 '2016_02',
 '2016_03',
 '2016_04',
 '2016_05',
 '2016_06',
 '2016_07',
 '2016_08',
 '2016_09',
 '2016_10',
 '2016_11',
 '2016_12',
 '2017_01',
 '2017_02',
 '2017_03',
 '2017_04',
 '2017_05',
 '2017_06',
 '2017_07',
 '2017_08',
 '2017_09',
 '2017_10',
 '2017_11',
 '2017_12',
 '2018_01',
 '2018_02',
 '2018_03',
 '2018_04',
 '2018_05',
 '2018_06',
 '2018_07',
 '2018_08',
 '2018_09',
 '2018_10',
 '2018_11',
 '2018_12',
 '2019_01',
 '2019_02',
 '2019_03',
 '2019_04',
 '2019_05',
 '2019_06',
 '2019_07',
 '2019_08',
 '2019_09',
 '2019_10',
 '2019_11',
 '2019_12',
 '2020_01',
 '2020_02',
 '2020_03',
 '2020_04',
 '2020_05',
 '2020_06',
 '2020_07',
 '2020_08',
 '2020_09',
 '2020_10',
 '2020_11',
 '2020_12',
 '2021_01',
 '2021_02',
 '2021_03',
 '2021_04',
 '2021_05',
 '2021_06',
 '2021_07',
 '2021_08',
 '2021_09',
 '2021_10',
 '2021_11',
 '2021_12',
 '2022_01',
 '2022_02',
 '2022_03',
 '20

In [16]:
NDMWD = []  #list()  # North Dakota Monthly Well (NDMWD) Data download process ~ 10 minutes
for month in myList:
    target_url = website + month + '.xlsx'
    print(target_url)
    r = requests.get(target_url)
    wb = pd.read_excel(r.content)
    NDMWD.append(wb)

NDMWD = pd.concat(NDMWD)

https://www.dmr.nd.gov/oilgas/mpr/2015_05.xlsx


<ipython-input-16-474bd43b1c0f>:6: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  wb = pd.read_excel(r.content)


https://www.dmr.nd.gov/oilgas/mpr/2015_06.xlsx


<ipython-input-16-474bd43b1c0f>:6: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  wb = pd.read_excel(r.content)


https://www.dmr.nd.gov/oilgas/mpr/2015_07.xlsx


...

<ipython-input-16-474bd43b1c0f>:6: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  wb = pd.read_excel(r.content)


https://www.dmr.nd.gov/oilgas/mpr/2024_06.xlsx


<ipython-input-16-474bd43b1c0f>:6: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  wb = pd.read_excel(r.content)


https://www.dmr.nd.gov/oilgas/mpr/2024_07.xlsx


<ipython-input-16-474bd43b1c0f>:6: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  wb = pd.read_excel(r.content)


In [17]:
#Create ND monthly well data table in csv format, which can be downloaded from folder on left hand side
NDMWD.to_csv('NDMWD.csv', index=False)

In [18]:
NDMWD.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1975906 entries, 0 to 21152
Data columns (total 21 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ReportDate  datetime64[ns]
 1   API_WELLNO  int64         
 2   FileNo      int64         
 3   Company     object        
 4   WellName    object        
 5   Quarter     object        
 6   Section     float64       
 7   Township    float64       
 8   Range       float64       
 9   County      object        
 10  FieldName   object        
 11  Pool        object        
 12  Oil         float64       
 13  Wtr         float64       
 14  Days        float64       
 15  Runs        int64         
 16  Gas         float64       
 17  GasSold     int64         
 18  Flared      float64       
 19  Lat         float64       
 20  Long        float64       
dtypes: datetime64[ns](1), float64(10), int64(4), object(6)
memory usage: 331.6+ MB


# **1.b Extract first month oil and gas produciton data, latitude and longitude**

In [19]:
#NDMWD = pd.read_csv('NDMWD.csv')
NDMWD.drop_duplicates(inplace=True)
NDMWD = NDMWD[NDMWD['Days'] >= 25].reset_index(drop=True)
NDMWD['ReportDate'] = pd.to_datetime(NDMWD['ReportDate'], errors = 'coerce')
NDMWD.tail(2)

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
1440514,2024-07-01,33009011910000,5815,EMPIRE NORTH DAKOTA LLC,RICE 23-12,NWSW,23.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,60.0,0.0,29.0,31,0.0,0,0.0,48.928322,-100.881995
1440515,2024-07-01,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,143.0,0.0,29.0,103,0.0,0,0.0,48.920808,-100.877109


In [20]:
ffmWells = NDMWD[['API_WELLNO', 'Pool','ReportDate']].groupby('API_WELLNO').min().rename(columns={'ReportDate':'ffMonth'}).reset_index()
ffmWells.tail(3)

,API_WELLNO,Pool,ffMonth
21570,33105061290000,BAKKEN,2024-07-01
21571,33105061300000,BAKKEN,2024-07-01
21572,33105903870000,DAKOTA,2018-01-01


In [21]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21573 entries, 0 to 21572
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  21573 non-null  int64         
 1   Pool        21573 non-null  object        
 2   ffMonth     21573 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 505.7+ KB


In [22]:
ffmWells = ffmWells.merge(NDMWD[['API_WELLNO','Pool','ReportDate','Oil','Gas','Wtr','Days','Runs', 'Lat', 'Long']], how='left', left_on=['API_WELLNO','Pool','ffMonth'], right_on = ['API_WELLNO','Pool','ReportDate'])
ffmWells.dropna(inplace=True)
ffmWells.reset_index(drop=True)
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,ReportDate,Oil,Gas,Wtr,Days,Runs,Lat,Long
21568,33105061220000,BAKKEN,2024-05-01,2024-05-01,7983.0,12315.0,15524.0,30.0,7907.0,48.340799,-103.075445
21569,33105061230000,BAKKEN,2024-05-01,2024-05-01,7290.0,11456.0,12890.0,29.0,7221.0,48.340799,-103.075260
21570,33105061290000,BAKKEN,2024-07-01,2024-07-01,17513.0,49910.0,17888.0,31.0,17530.0,48.281831,-103.014284
21571,33105061300000,BAKKEN,2024-07-01,2024-07-01,17754.0,67166.0,18273.0,31.0,17775.0,48.281831,-103.014148
21572,33105903870000,DAKOTA,2018-01-01,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385


In [23]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21542 entries, 0 to 21572
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  21542 non-null  int64         
 1   Pool        21542 non-null  object        
 2   ffMonth     21542 non-null  datetime64[ns]
 3   ReportDate  21542 non-null  datetime64[ns]
 4   Oil         21542 non-null  float64       
 5   Gas         21542 non-null  float64       
 6   Wtr         21542 non-null  float64       
 7   Days        21542 non-null  float64       
 8   Runs        21542 non-null  float64       
 9   Lat         21542 non-null  float64       
 10  Long        21542 non-null  float64       
dtypes: datetime64[ns](2), float64(7), int64(1), object(1)
memory usage: 2.0+ MB


In [24]:
ffmWells.drop('ReportDate', axis=1, inplace=True )
ffmWells['bpd'] =  ffmWells.Oil/ffmWells.Days
ffmWells['Mcfpd'] =  ffmWells.Gas/ffmWells.Days

In [25]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd
21568,33105061220000,BAKKEN,2024-05-01,7983.0,12315.0,15524.0,30.0,7907.0,48.340799,-103.075445,266.100000,410.500000
21569,33105061230000,BAKKEN,2024-05-01,7290.0,11456.0,12890.0,29.0,7221.0,48.340799,-103.075260,251.379310,395.034483
21570,33105061290000,BAKKEN,2024-07-01,17513.0,49910.0,17888.0,31.0,17530.0,48.281831,-103.014284,564.935484,1610.000000
21571,33105061300000,BAKKEN,2024-07-01,17754.0,67166.0,18273.0,31.0,17775.0,48.281831,-103.014148,572.709677,2166.645161
21572,33105903870000,DAKOTA,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385,13.225806,0.000000


In [26]:
ffmWells.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21542 entries, 0 to 21572
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   API_WELLNO  21542 non-null  int64         
 1   Pool        21542 non-null  object        
 2   ffMonth     21542 non-null  datetime64[ns]
 3   Oil         21542 non-null  float64       
 4   Gas         21542 non-null  float64       
 5   Wtr         21542 non-null  float64       
 6   Days        21542 non-null  float64       
 7   Runs        21542 non-null  float64       
 8   Lat         21542 non-null  float64       
 9   Long        21542 non-null  float64       
 10  bpd         21542 non-null  float64       
 11  Mcfpd       21542 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 2.1+ MB


# **1.c Wells latitude and longitude to MHB allocation**

In [27]:
# Maidenhead grid locators courtesy of Walter Underwood K6WRU:
# https://ham.stackexchange.com/questions/221/how-can-one-convert-from-lat-long-to-grid-square


In [28]:
upper = 'ABCDEFGHIJKLMNOPQRSTUVWX'
lower = 'abcdefghijklmnopqrstuvwx'

In [29]:
def gridCalc(row):
    dec_lat = row['Lat']
    dec_lon = row['Long']
    if not (-180<=dec_lon<180):
        sys.stderr.write('longitude must be -180<=lon<180, given %f\n'%dec_lon)
        sys.exit(32)
    if not (-90<=dec_lat<90):
        sys.stderr.write('latitude must be -90<=lat<90, given %f\n'%dec_lat)
        sys.exit(33) # can't handle north pole, sorry, [A-R]

    adj_lat = dec_lat + 90.0
    adj_lon = dec_lon + 180.0

    grid_lat_sq = upper[int(adj_lat/10)];
    grid_lon_sq = upper[int(adj_lon/20)];

    grid_lat_field = str(int(adj_lat%10))
    grid_lon_field = str(int((adj_lon/2)%10))

    adj_lat_remainder = (adj_lat - int(adj_lat)) * 60
    adj_lon_remainder = ((adj_lon) - int(adj_lon/2)*2) * 60

    grid_lat_subsq = lower[int(adj_lat_remainder/2.5)]
    grid_lon_subsq = lower[int(adj_lon_remainder/5)]

    return grid_lon_sq + grid_lat_sq + grid_lon_field + grid_lat_field + grid_lon_subsq + grid_lat_subsq

In [30]:
ffmWells['MHB'] = ffmWells.apply(gridCalc, axis=1)

In [31]:
ffmWells.tail()

,API_WELLNO,Pool,ffMonth,Oil,Gas,Wtr,Days,Runs,Lat,Long,bpd,Mcfpd,MHB
21568,33105061220000,BAKKEN,2024-05-01,7983.0,12315.0,15524.0,30.0,7907.0,48.340799,-103.075445,266.100000,410.500000,DN88li
21569,33105061230000,BAKKEN,2024-05-01,7290.0,11456.0,12890.0,29.0,7221.0,48.340799,-103.075260,251.379310,395.034483,DN88li
21570,33105061290000,BAKKEN,2024-07-01,17513.0,49910.0,17888.0,31.0,17530.0,48.281831,-103.014284,564.935484,1610.000000,DN88lg
21571,33105061300000,BAKKEN,2024-07-01,17754.0,67166.0,18273.0,31.0,17775.0,48.281831,-103.014148,572.709677,2166.645161,DN88lg
21572,33105903870000,DAKOTA,2018-01-01,410.0,0.0,0.0,31.0,946.0,48.404169,-103.475385,13.225806,0.000000,DN88gj


In [32]:
ffmWells.to_csv('ffmWells.csv', index=False)

# **2.a Download well bore data from ND DMR ~5min**

In [33]:
!pip install dbfread

In [34]:
import pandas as pd
import plotly.express as px
from dbfread import DBF
import ipywidgets as widgets
import csv
from math import radians, sin, cos, sqrt, atan2

In [35]:
# run command to download data.  In case it is not able to download, open the link and upload the data to this notebook from left hand side menu.
!wget https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip

--2024-10-16 20:13:45--  https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip
Resolving gis.dmr.nd.gov (gis.dmr.nd.gov)... 165.234.156.197
Connecting to gis.dmr.nd.gov (gis.dmr.nd.gov)|165.234.156.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330471001 (315M) [application/x-zip-compressed]
Saving to: ‘OGD_Horizontals.zip’

OGD_Horizontals.zip 100%[===================>] 315.16M  36.9MB/s    in 8.8s    

2024-10-16 20:13:54 (35.9 MB/s) - ‘OGD_Horizontals.zip’ saved [330471001/330471001]



In [36]:
!unzip OGD_Horizontals.zip

Archive:  OGD_Horizontals.zip
  inflating: OGD_Horizontals.cpg     
  inflating: OGD_Horizontals.dbf     
  inflating: OGD_Horizontals.prj     
  inflating: OGD_Horizontals.sbn     
  inflating: OGD_Horizontals.sbx     
  inflating: OGD_Horizontals.shp     
  inflating: OGD_Horizontals.shp.xml  
  inflating: OGD_Horizontals.shx     
  inflating: OGD_Horizontals_Line.cpg  
  inflating: OGD_Horizontals_Line.dbf  
  inflating: OGD_Horizontals_Line.prj  
  inflating: OGD_Horizontals_Line.sbn  
  inflating: OGD_Horizontals_Line.sbx  
  inflating: OGD_Horizontals_Line.shp  
  inflating: OGD_Horizontals_Line.shp.xml  
  inflating: OGD_Horizontals_Line.shx  


In [37]:
# to write OGD_Horizontals.dbf to csv file ~ 2 minutes
filename= 'OGD_Horizontals.dbf'
table = DBF(filename)
f=open('ndWells.csv', 'w')
writer = csv.writer(f)
writer.writerow(table.field_names)
for record in table:
  writer.writerow(list(record.values()))

In [38]:
dfWells = pd.read_csv('ndWells.csv')
dfWells.head(3)

,wl_permit,api_wellno,api_format,long,lat,well_sub,measdpth,inclinatio,azimuth,tvd,coordns,coordnsdir,coordew,coordewdir,surveytype
0,1741,33007000260000,33-007-00026-00-00,-103.392973,46.866931,LAT1,9000.0,0.77,4.33,8999.030273,38.310001,N,35.090000,W,STP
1,1741,33007000260000,33-007-00026-00-00,-103.392971,46.866931,LAT1,9015.0,3.30,88.00,9014.021484,38.425602,N,34.650803,W,SPT
2,1741,33007000260000,33-007-00026-00-00,-103.392969,46.866931,LAT1,9022.0,5.30,88.00,9021.000977,38.443920,N,34.126297,W,SPT


# **2.b Filter out first laterals and estimate lateral length**

In [39]:
dfWells['myLateral'] = ['1stLat' if x=='LAT1' else '1stLat' if x=='STK1' else 'VERT' if x=='VERT' else 'othLat' for x in dfWells['well_sub']]
dfWells.tail(3)

,wl_permit,api_wellno,api_format,long,lat,well_sub,measdpth,inclinatio,azimuth,tvd,coordns,coordnsdir,coordew,coordewdir,surveytype,myLateral
5133571,90466,33105904660000,33-105-90466-00-00,-103.411652,48.287815,VERT,4796.0,0.3,333.500000,4795.767578,5.322332,S,13.826293,W,SPT,VERT
5133572,90466,33105904660000,33-105-90466-00-00,-103.411652,48.287816,VERT,4891.0,0.1,346.399994,4890.767090,5.019176,S,13.956761,W,SPT,VERT
5133573,90466,33105904660000,33-105-90466-00-00,-103.411652,48.287817,VERT,4986.0,0.1,353.799988,4985.767090,4.856180,S,13.985209,W,SPT,VERT


In [40]:
flWells = dfWells.groupby(['api_wellno','myLateral']).agg(latMin=('lat', 'min'),latMax=('lat', 'max'),
                                                        longMin=('long', 'min'),longMax=('long', 'max'),
                                                        tvdMin=('tvd', 'min'),tvdMax=('tvd', 'max')).reset_index()

In [41]:
def latft(latMin, latMax, longMin, longMax):

    R = 20925721

    lat1 = radians(latMin)
    lon1 = radians(longMin)
    lat2 = radians(latMax)
    lon2 = radians(longMax)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    #print("latL_km:", distance)
    return distance

In [42]:
flWells['totll'] = flWells.apply(lambda row: latft(row['latMin'],row['latMax'],row['longMin'],row['longMax']),  axis=1)

In [43]:
flWellsOnly = flWells[['api_wellno','myLateral','totll']][flWells['myLateral'] == '1stLat'].reset_index(drop=True)
flWellsOnly.tail()

,api_wellno,myLateral,totll
20960,33105062550000,1stLat,15962.575052
20961,33105062650000,1stLat,10680.149038
20962,33105902990000,1stLat,2102.817016
20963,33105904500000,1stLat,1787.417830
20964,33105904660000,1stLat,2305.694952


In [44]:
flWellsOnly[['api_wellno','myLateral','totll']].to_csv('flWellsOnly.csv', index=False)

In [45]:
flWells[flWells['api_wellno']==33105040690000]

,api_wellno,myLateral,latMin,latMax,longMin,longMax,tvdMin,tvdMax,totll
38968,33105040690000,1stLat,48.282229,48.282416,-102.980406,-102.979728,7251.502930,8439.927734,178.459657
38969,33105040690000,VERT,48.282314,48.282738,-102.983372,-102.979511,1768.457642,8985.621094,951.301084


In [46]:
## 3D plot of specific well to verify estimation
plWell = dfWells[['api_wellno','lat','long','tvd','well_sub']][dfWells['api_wellno'] == 33105040690000]

In [47]:
fig = px.scatter_3d(plWell, x='long', y='lat', z='tvd', color='well_sub')
fig.update_traces(marker_size = 3)
fig.update_scenes(yaxis_autorange="reversed")
fig.update_scenes(xaxis_autorange="reversed")
fig.update_scenes(zaxis_range=[16000, 0])
fig.update_layout({
    'plot_bgcolor': 'rgba(100, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                      })

# **3.a Download and unzip FracFocus.org well completion data ~5min**

In [48]:
import pandas as pd
import requests
import zipfile
import io
import datetime

In [49]:
link2FFcsvZip = 'http://fracfocusdata.org/digitaldownload/FracFocusCSV.zip'

In [50]:
request = requests.get(link2FFcsvZip)
zipBag = zipfile.ZipFile(io.BytesIO(request.content))

In [51]:
zipFiles = zipBag.namelist()
zipFiles

['DisclosureList_1.csv',
 'FracFocusRegistry_1.csv',
 'FracFocusRegistry_10.csv',
 'FracFocusRegistry_11.csv',
 'FracFocusRegistry_12.csv',
 'FracFocusRegistry_13.csv',
 'FracFocusRegistry_14.csv',
 'FracFocusRegistry_2.csv',
 'FracFocusRegistry_3.csv',
 'FracFocusRegistry_4.csv',
 'FracFocusRegistry_5.csv',
 'FracFocusRegistry_6.csv',
 'FracFocusRegistry_7.csv',
 'FracFocusRegistry_8.csv',
 'FracFocusRegistry_9.csv',
 'readme csv.txt',
 'WaterSource_1.csv']

In [52]:
#Unzip files and create corresponding dataframes
RegistryUpload_files=[]
FracFocusRegistry_files=[]

In [53]:
for file_name in zipFiles:
    if (('DisclosureList' in file_name)) & (file_name.endswith('.csv')):
        RegistryUpload_files.append(file_name)
    if (('FracFocusRegistry' in file_name)) & (file_name.endswith('.csv')):
        FracFocusRegistry_files.append(file_name)

In [54]:
RegistryUpload_files

['DisclosureList_1.csv']

In [55]:
dfs = list()
for file_name in RegistryUpload_files:
    df = pd.read_csv(zipBag.open(file_name), low_memory=False, dtype={'APINumber': 'str','StateNumber': 'str', 'CountyNumber': 'str'})
    dfs.append(df)

RegistryUpload = pd.concat(dfs, axis=0, ignore_index=True)
RegistryUpload.head(2)

,DisclosureId,JobStartDate,JobEndDate,APINumber,StateName,CountyName,OperatorName,WellName,Latitude,Longitude,Projection,TVD,TotalBaseWaterVolume,TotalBaseNonWaterVolume,FFVersion,FederalWell,IndianWell
0,02cd05a7-4b73-4722-9fba-1f798842a879,5/1/1955 12:00:00 AM,5/1/1955 12:00:00 AM,42317372620000,Texas,Martin,Pioneer Natural Resources,Rogers 42 #5,32.283431,-101.906575,NAD27,NaN,NaN,NaN,1,False,False
1,416d8b17-822f-4743-8c79-baf015a6de24,5/19/1982 12:00:00 AM,5/19/1982 12:00:00 AM,49009219470000,Wyoming,Converse,"Chesapeake Operating, Inc.",WILLIAM VALENTINE 1,42.972810,-105.953840,NAD27,NaN,NaN,NaN,1,False,False


In [56]:
RegistryUpload.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226155 entries, 0 to 226154
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DisclosureId             226155 non-null  object 
 1   JobStartDate             226140 non-null  object 
 2   JobEndDate               226155 non-null  object 
 3   APINumber                226155 non-null  object 
 4   StateName                226155 non-null  object 
 5   CountyName               226151 non-null  object 
 6   OperatorName             226155 non-null  object 
 7   WellName                 226155 non-null  object 
 8   Latitude                 226120 non-null  float64
 9   Longitude                226120 non-null  float64
 10  Projection               226155 non-null  object 
 11  TVD                      196015 non-null  float64
 12  TotalBaseWaterVolume     195986 non-null  float64
 13  TotalBaseNonWaterVolume  175828 non-null  float64
 14  FFVe

In [57]:
FracFocusRegistry_files

['FracFocusRegistry_1.csv',
 'FracFocusRegistry_10.csv',
 'FracFocusRegistry_11.csv',
 'FracFocusRegistry_12.csv',
 'FracFocusRegistry_13.csv',
 'FracFocusRegistry_14.csv',
 'FracFocusRegistry_2.csv',
 'FracFocusRegistry_3.csv',
 'FracFocusRegistry_4.csv',
 'FracFocusRegistry_5.csv',
 'FracFocusRegistry_6.csv',
 'FracFocusRegistry_7.csv',
 'FracFocusRegistry_8.csv',
 'FracFocusRegistry_9.csv']

In [58]:
dfs = list()
for file_name in FracFocusRegistry_files:
    df = pd.read_csv(zipBag.open(file_name), low_memory=False, dtype={'APINumber': 'str','StateNumber': 'str', 'CountyNumber': 'str'})
    dfs.append(df)

FracFocusRegistry = pd.concat(dfs, axis=0, ignore_index=True)
FracFocusRegistry.head(2)

,DisclosureId,JobStartDate,JobEndDate,APINumber,StateName,CountyName,OperatorName,WellName,Latitude,Longitude,...,IngredientsId,CASNumber,IngredientName,IngredientCommonName,PercentHighAdditive,PercentHFJob,IngredientComment,IngredientMSDS,MassIngredient,ClaimantCompany
0,02cd05a7-4b73-4722-9fba-1f798842a879,5/1/1955 12:00:00 AM,5/1/1955 12:00:00 AM,42317372620000,Texas,Martin,Pioneer Natural Resources,Rogers 42 #5,32.283431,-101.906575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,416d8b17-822f-4743-8c79-baf015a6de24,5/19/1982 12:00:00 AM,5/19/1982 12:00:00 AM,49009219470000,Wyoming,Converse,"Chesapeake Operating, Inc.",WILLIAM VALENTINE 1,42.972810,-105.953840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
FracFocusRegistry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6620179 entries, 0 to 6620178
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   DisclosureId             object 
 1   JobStartDate             object 
 2   JobEndDate               object 
 3   APINumber                object 
 4   StateName                object 
 5   CountyName               object 
 6   OperatorName             object 
 7   WellName                 object 
 8   Latitude                 float64
 9   Longitude                float64
 10  Projection               object 
 11  TVD                      float64
 12  TotalBaseWaterVolume     float64
 13  TotalBaseNonWaterVolume  float64
 14  FFVersion                int64  
 15  FederalWell              bool   
 16  IndianWell               bool   
 17  PurposeId                object 
 18  TradeName                object 
 19  Supplier                 object 
 20  Purpose                  object 
 21  Ingredie

# **3.b Estimate water and sand used in copletions of ND wells only**

In [72]:
# Filter out ND wells and list water used in completions
ffWellsND = RegistryUpload[['DisclosureId','APINumber','JobEndDate','TotalBaseWaterVolume']][RegistryUpload['StateName']== 'North Dakota'].reset_index(drop=True)
ffWellsND['JobEndDate'] = pd.to_datetime(ffWellsND['JobEndDate'], errors = 'coerce').dt.date
ffWellsND.tail()

<ipython-input-72-fdf7205ff525>:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,DisclosureId,APINumber,JobEndDate,TotalBaseWaterVolume
17135,5d9857a2-edb7-434d-94bb-42b50aa2c53e,33025048690000,2024-09-30,6296934.0
17136,81299cdf-b2d2-4a7c-a018-0114275c53ca,33025048720000,2024-09-30,6292886.0
17137,1e988afd-5131-4cdd-8c39-615fcae63d31,33025048680000,2024-10-01,6695446.0
17138,abef67a4-01f7-420a-91c8-2cc481e53c7c,33025048700000,2024-10-01,6996033.0
17139,aaac8d19-1cf5-484a-944d-eaa63ebc58e8,33025048710000,2024-10-01,6980867.0


In [75]:
#Filter out sand used during completions
swData = FracFocusRegistry[['DisclosureId','APINumber','JobEndDate','IngredientName','Purpose','TradeName','MassIngredient','PercentHFJob','IngredientsId','TotalBaseWaterVolume']][FracFocusRegistry['StateName']== 'North Dakota'].reset_index(drop=True)
swData['JobEndDate'] = pd.to_datetime(swData['JobEndDate'], errors = 'coerce').dt.date

<ipython-input-75-0f1052939e7c>:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [76]:
tnWords = ['sand','mesh','proppant','ceramic','quartz','silica','crystalline']
swDataFilter = swData[swData['TradeName'].str.contains('|'.join(tnWords), case=False, na=False, regex = True)].copy().reset_index(drop=True)
swDataFilter.tail(2)

,DisclosureId,APINumber,JobEndDate,IngredientName,Purpose,TradeName,MassIngredient,PercentHFJob,IngredientsId,TotalBaseWaterVolume
46093,f7f47f67-fe99-4b6d-8a7f-5b9f71f3af88,33025035890000,2019-03-22,NaN,Propping Agent,Sand (Proppant),NaN,NaN,NaN,11385210.48
46094,37726bdb-7aee-447d-8597-55bd64e155c0,33025035900000,2019-03-22,NaN,Propping Agent,Sand (Proppant),NaN,NaN,NaN,11864286.00


In [77]:
exclusionList = ['water','transport','acid','stabilizer','carrier','clay','ammonium','chloride','Potassium','KCL','chlorine','naphtha','petroleum','aromatic','surfactant','alcohol','gum','tetrahydrate','carbonate','glycol']
swDataFilter = swDataFilter[~swDataFilter['TradeName'].str.contains('|'.join(exclusionList), case=False, na=False)].copy().reset_index(drop=True)
swDataFilter.tail(2)

,DisclosureId,APINumber,JobEndDate,IngredientName,Purpose,TradeName,MassIngredient,PercentHFJob,IngredientsId,TotalBaseWaterVolume
26967,f7f47f67-fe99-4b6d-8a7f-5b9f71f3af88,33025035890000,2019-03-22,NaN,Propping Agent,Sand (Proppant),NaN,NaN,NaN,11385210.48
26968,37726bdb-7aee-447d-8597-55bd64e155c0,33025035900000,2019-03-22,NaN,Propping Agent,Sand (Proppant),NaN,NaN,NaN,11864286.00


In [78]:
#Estimate mass from PercentHFJob
swDataFilter['derWeight'] = swDataFilter['TotalBaseWaterVolume']*.0935*swDataFilter['PercentHFJob']

In [79]:
def weightCondition(s):
    if (s['MassIngredient'] > 0):
        return s['MassIngredient']
    else:
        return s['derWeight']

In [80]:
swDataFilter['bestWeight'] = swDataFilter.apply(weightCondition, axis=1)

In [82]:
SandUsed = swDataFilter[['DisclosureId','APINumber','JobEndDate','bestWeight']].groupby(['DisclosureId','APINumber','JobEndDate']).sum().rename(columns={'bestWeight':'SandLB'}).reset_index()

In [83]:
SandUsed = SandUsed[SandUsed['SandLB'] > 0]
SandUsed.reset_index(drop=True, inplace=True)
SandUsed.tail()

,DisclosureId,APINumber,JobEndDate,SandLB
7881,ffd1e150-b72e-472b-9c9a-40617cbf77ff,33053099340000,2023-07-01,1.018748e+07
7882,ffda59f2-0e93-47d7-935e-7185225fdb96,33053082770000,2018-12-01,1.430503e+06
7883,ffe3b8ae-188c-4534-8173-aa7091427cd3,33061019950000,2013-07-11,3.354627e+06
7884,fff6681f-09ff-48f3-9479-a1d9859a7deb,33053084650000,2019-02-19,1.142050e+07
7885,fffc4f8a-2730-4e25-beaf-3b0fcde32cbc,33053071600000,2017-04-28,5.908000e+04


In [86]:
#Merge water and sand used during completions
ffWellsND = ffWellsND.merge(SandUsed[['DisclosureId','SandLB']], how='left', left_on = 'DisclosureId', right_on = 'DisclosureId' )

In [90]:
ffWellsND.tail()

,APINumber,JobEndDate,TotalBaseWaterVolume,SandLB
17135,33025048690000,2024-09-30,6296934.0,6.595012e+06
17136,33025048720000,2024-09-30,6292886.0,6.595000e+06
17137,33025048680000,2024-10-01,6695446.0,1.641890e+08
17138,33025048700000,2024-10-01,6996033.0,1.252137e+07
17139,33025048710000,2024-10-01,6980867.0,3.085354e+07


In [89]:
ffWellsND.drop(['DisclosureId'], axis=1, inplace=True)

In [91]:
#Filter out first frac jobs
ffWellsND = ffWellsND.sort_values(['APINumber','JobEndDate']).reset_index(drop = True)
ffWellsND["sfID"] = ffWellsND.groupby(['APINumber']).cumcount()
ffWellsND = ffWellsND[ffWellsND['sfID']==0].reset_index(drop = True)  # filters out refrac jobs, keeps only the first frac jobs
ffWellsND.drop('sfID', inplace=True, axis=1)
ffWellsND.rename({'TotalBaseWaterVolume': 'WaterGal'}, axis=1, inplace=True)
ffWellsND["APINumber"] = ffWellsND["APINumber"].astype(int)
ffWellsND.tail()

,APINumber,JobEndDate,WaterGal,SandLB
16093,33105121850000,2012-02-24,NaN,NaN
16094,33610338000000,2022-06-08,13496802.0,10357262.37
16095,33610338100000,2022-06-09,14820967.0,11223894.87
16096,35023006670000,2011-08-27,NaN,NaN
16097,49037287730000,2012-07-10,2339813.0,NaN


In [92]:
ffWellsND.to_csv('ffWellsND.csv', index=False)
ffWellsND.tail()

,APINumber,JobEndDate,WaterGal,SandLB
16093,33105121850000,2012-02-24,NaN,NaN
16094,33610338000000,2022-06-08,13496802.0,10357262.37
16095,33610338100000,2022-06-09,14820967.0,11223894.87
16096,35023006670000,2011-08-27,NaN,NaN
16097,49037287730000,2012-07-10,2339813.0,NaN


# **4. Combine files and write data file for Sub-county analysis**

In [93]:
# In disagregated form, read the files from previously derived csv files
#ffmWells = pd.read_csv('ffmWells.csv')
#flWellsOnly = pd.read_csv('flWellsOnly.csv')
#ffWellsND = pd.read_csv('ffWellsND.csv')

In [94]:
ND_Data = ffmWells[['API_WELLNO','Pool','ffMonth','bpd','Mcfpd','MHB']].merge(flWellsOnly, how = 'left', left_on = 'API_WELLNO', right_on = 'api_wellno')
ND_Data.drop(['api_wellno','myLateral'], axis=1, inplace=True)
ND_Data.columns = ['API14', 'Pool', 'feDate', 'maxPo', 'maxPg','MHB','LL']
ND_Data.tail()

,API14,Pool,feDate,maxPo,maxPg,MHB,LL
21537,33105061220000,BAKKEN,2024-05-01,266.100000,410.500000,DN88li,15619.042759
21538,33105061230000,BAKKEN,2024-05-01,251.379310,395.034483,DN88li,15644.364532
21539,33105061290000,BAKKEN,2024-07-01,564.935484,1610.000000,DN88lg,10671.844083
21540,33105061300000,BAKKEN,2024-07-01,572.709677,2166.645161,DN88lg,10674.552562
21541,33105903870000,DAKOTA,2018-01-01,13.225806,0.000000,DN88gj,NaN


In [95]:
ND_Data = ND_Data.merge(ffWellsND, how = 'left', left_on = 'API14', right_on = 'APINumber')
ND_Data.tail()

,API14,Pool,feDate,maxPo,maxPg,MHB,LL,APINumber,JobEndDate,WaterGal,SandLB
21537,33105061220000,BAKKEN,2024-05-01,266.100000,410.500000,DN88li,15619.042759,3.310506e+13,2024-04-19,14870126.0,NaN
21538,33105061230000,BAKKEN,2024-05-01,251.379310,395.034483,DN88li,15644.364532,3.310506e+13,2024-04-20,15233581.0,NaN
21539,33105061290000,BAKKEN,2024-07-01,564.935484,1610.000000,DN88lg,10671.844083,3.310506e+13,2024-02-26,7790059.0,NaN
21540,33105061300000,BAKKEN,2024-07-01,572.709677,2166.645161,DN88lg,10674.552562,3.310506e+13,2024-02-26,7730547.0,NaN
21541,33105903870000,DAKOTA,2018-01-01,13.225806,0.000000,DN88gj,NaN,NaN,NaN,NaN,NaN


In [96]:
#Drop wells with missing values
ND_Data.dropna(inplace=True)
ND_Data.reset_index(inplace=True, drop=True)
ND_Data.tail()

,API14,Pool,feDate,maxPo,maxPg,MHB,LL,APINumber,JobEndDate,WaterGal,SandLB
7225,33105060490000,BAKKEN,2024-03-01,385.483871,1445.612903,DN88ln,10748.670579,3.310506e+13,2023-11-06,7447573.0,10115764.11
7226,33105060500000,BAKKEN,2024-03-01,337.193548,1371.225806,DN88ln,10745.854055,3.310506e+13,2023-11-06,7566559.0,10114974.90
7227,33105060510000,BAKKEN,2024-03-01,450.310345,1318.413793,DN88ln,10717.311777,3.310506e+13,2023-11-02,7912265.0,10115174.70
7228,33105060520000,BAKKEN,2024-03-01,314.290323,875.387097,DN88ln,10730.701064,3.310506e+13,2023-11-02,7987943.0,10114924.95
7229,33105061140000,BAKKEN,2024-02-01,963.206897,1421.310345,DN88oh,9899.942026,3.310506e+13,2023-12-22,8083362.0,9652284.00


In [97]:
ND_Bakken = ND_Data[ND_Data['Pool']=='BAKKEN']  #Filter out Bakken wells only
ND_Bakken = ND_Bakken[ND_Bakken['feDate']>'2015-06-01'].reset_index(drop = True)  #Filter out first two month to eliminate wells that started before data is available
ND_Bakken

,API14,Pool,feDate,maxPo,maxPg,MHB,LL,APINumber,JobEndDate,WaterGal,SandLB
0,33007017390000,BAKKEN,2018-03-01,21.400000,10.760000,DN87kg,9988.611253,3.300702e+13,2013-06-14,1519434.0,100331.00
1,33007017630000,BAKKEN,2015-07-01,243.903226,384.000000,DN86iw,11573.584346,3.300702e+13,2015-01-30,8957550.0,5276217.00
2,33007017970000,BAKKEN,2015-08-01,1.833333,8.300000,DN87ke,9962.164000,3.300702e+13,2013-09-01,422625.0,222478.00
3,33007018050000,BAKKEN,2018-12-01,150.066667,110.066667,DN87fb,9701.463543,3.300702e+13,2018-07-09,92042880.0,2330112.36
4,33007018060000,BAKKEN,2015-07-01,76.517241,57.172414,DN87kf,10134.729658,3.300702e+13,2013-12-11,2640162.0,1555981.00
...,...,...,...,...,...,...,...,...,...,...,...
3843,33105060490000,BAKKEN,2024-03-01,385.483871,1445.612903,DN88ln,10748.670579,3.310506e+13,2023-11-06,7447573.0,10115764.11
3844,33105060500000,BAKKEN,2024-03-01,337.193548,1371.225806,DN88ln,10745.854055,3.310506e+13,2023-11-06,7566559.0,10114974.90
3845,33105060510000,BAKKEN,2024-03-01,450.310345,1318.413793,DN88ln,10717.311777,3.310506e+13,2023-11-02,7912265.0,10115174.70
3846,33105060520000,BAKKEN,2024-03-01,314.290323,875.387097,DN88ln,10730.701064,3.310506e+13,2023-11-02,7987943.0,10114924.95


In [98]:
ND_Bakken.to_excel('ND_BakkenPublic.xlsx', sheet_name='ND_BakkenPublic', index=False)

In [99]:
#download the file
from google.colab import files
files.download('ND_BakkenPublic.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>